# 리트리버가 가져온 정답이 없는 문서를 MRC가 거를 수 있도록, 데이터를 제작하자.

답이 없는 문서에 대해서, 좀 더 낮은 logit을 출력하는 것을 목표로 했습니다.

리트리버가 가져온 문서들의 스코어 비교가 더 잘되게 하기 위함입니다.

성능 향상 여부는 그때그때 달랐습니다. 너무 많은 샘플을 주면 수렴 자체가 어렵습니다.

---



엘라스틱서치의 설정에 따라 다른 문서를 가져올 수 있습니다.

본 설정은, 김남혁 캠퍼님께 받았던 설정을 기반으로 합니다.

## 참고자료

http://boostcamp.stages.ai/competitions/31/discussion/post/314

https://colab.research.google.com/github/neuml/txtai/blob/master/examples/04_Add_semantic_search_to_Elasticsearch.ipynb#scrollTo=0y1UA4-q-YdA

https://jvvp.tistory.com/1152

https://m.blog.naver.com/PostView.nhn?blogId=slykid&logNo=221559872279&proxyReferer=https:%2F%2Fwww.google.com%2F

In [ ]:
!pip install datasets==1.6.2
!python -m pip install elasticsearch
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.2
!pip install pororo

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/content/elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
! sleep 30

In [ ]:
! /content/elasticsearch-7.9.2/bin/elasticsearch-plugin install analysis-nori

-> Installing analysis-nori
-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Failed installing analysis-nori
-> Rolling back analysis-nori
-> Rolled back analysis-nori
ERROR: plugin directory [/content/elasticsearch-7.9.2/plugins/analysis-nori] already exists; if you need to update the plugin, uninstall it first using command 'remove analysis-nori'


In [ ]:
es_server.kill()

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['/content/elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
! sleep 30

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200", timeout=30, max_retries=10, retry_on_timeout=True)

#출처: https://somjang.tistory.com/entry/Elasticsearch-ConnectionTimeout-caused-by-ReadTimeoutError-feat-bulk-API-python [솜씨좋은장씨]

In [ ]:
es.info()

{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'r35-ge6PRiK4KBeF1oFXwg',
 'name': '6dac72d0c4dd',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2020-09-23T00:45:33.626720Z',
  'build_flavor': 'default',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.6.2',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.8.0',
  'number': '7.9.2'}}

In [ ]:
es.indices.create(index = 'document_a',
                  body = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      'synonyms':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "nori_number",
                                                 "cjk_bigram",
                                                 "decimal_digit",
                                                 "stemmer",
                                                 "trim"]
                                  }
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'title':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              },
                              'text':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              },
                              'text_origin':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              }
                          }
                      }
                  }
                  )

In [ ]:
import json
import pandas as pd
import re
from tqdm import tqdm
from pororo import Pororo
from datasets import load_from_disk
from elasticsearch import Elasticsearch, helpers
from datasets import Dataset

In [ ]:
wiki_path = #'/content/drive/MyDrive/Colab_data/ai_boostcamp_data/p-stage-3/title_고려안함/wiki_for_retriever/new_wikipedia_documents.json'
path = #'/content/drive/MyDrive/Colab_data/ai_boostcamp_data/p-stage-3/title_고려안함/datasets1.6.2/'

In [ ]:
with open(os.path.join(wiki_path), "r") as f:
    wiki_data = pd.DataFrame(json.load(f)).transpose()

In [ ]:
# 사용할 데이터셋을 가져와주세요.
tdk12_dataset = load_from_disk(path+'tdk12')

In [ ]:
wiki_data = wiki_data.drop_duplicates(['text']) # 3876

wiki_data = wiki_data.reset_index()

del wiki_data['index']

In [ ]:
wiki_data['text_origin'] = wiki_data['text']

wiki_data['text_origin'] = wiki_data['text_origin'].apply(lambda x : ' '.join(re.sub(r'''[^ \r\nㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9ぁ-ゔァ-ヴー々〆〤一-龥~₩!@#$%^&*()“”‘’《》≪≫〈〉『』「」＜＞_+|{}:"<>?`\-=\\[\];',.\/·]''', ' ', str(x.lower().strip())).split()))

wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\\n\\n',' '))
wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\n\n',' '))
wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\\n',' '))
wiki_data['text'] = wiki_data['text'].apply(lambda x : x.replace('\n',' '))

wiki_data['text'] = wiki_data['text'].apply(lambda x : ' '.join(re.sub(r'''[^ \r\nㄱ-ㅎㅏ-ㅣ가-힣]''', ' ', str(x.lower().strip())).split()))

In [ ]:
title = []
text = []
text_origin = []

for num in tqdm(range(len(wiki_data))):
    cnt = 0
    while cnt < len(wiki_data['text'][num]):
        title.append(wiki_data['title'][num])
        text.append(wiki_data['text'][num][cnt:cnt+1000])
        text_origin.append(wiki_data['text_origin'][num])
        cnt+=1000

In [ ]:
df = pd.DataFrame({'title':title,'text':text,'text_origin':text_origin})

In [ ]:
buffer = []
rows = 0

for num in tqdm(range(len(df))):
    article = {"_id": num,
               "_index": "document_a", 
               "title" : df['title'][num],
               "text" : df['text'][num],
               "text_origin" : df['text_origin'][num]}

    buffer.append(article)

    rows += 1

    if rows % 1000 == 0:
        helpers.bulk(es, buffer)
        buffer = []

        print("Inserted {} articles".format(rows), end="\r")

if buffer:
    helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))

In [ ]:
ner = Pororo(task="ner", lang="ko")

# 세팅이 끝났으니 생성하자.

## 정답이 아예 없는 오답 문서-질문 데이터 제작

특정 질문에 대한 답이 없다는 것을 제공하는 데이터입니다.

정답이 문서에 들어가 있는 경우에는, 해당 문서를 가져오지 않는 방법입니다.

또한, 가져온 문서들는 1000 글자까지만 받아 concat 합니다. 학습 시간이 부족해서 그랬습니다.

생성된 데이터는 answers가 없고, 답을 이끌어 낼 수 없는 지문만이 context에 제공됩니다.

In [ ]:
def full_neg_2_concat(dict_):
    query = {
        'query':{
            'bool':{
                'must':[
                        {'match':{'text':dict_['question']}}
                ],
                'should':[
                        {'match':{'text':' '.join([i[0] for i in ner(dict_['question']) if i[1] != 'O'])}}
                ]
            }
        }
    }
    es_docs = es.search(index='document_a',body=query,size=20)['hits']['hits']
    
    docs = []
    for doc in es_docs:
        d = doc['_source']['text_origin']
        if dict_['context'] == d:
            continue
        if dict_['answers']['text'][0] in d: # 가져온 문서에 정답 span이 존재하지 않아야 한다.
            continue
        docs.append(d[:1000])
        if len(docs) == 2:
            break

    if not docs:
        docs.append('') # 차후에 이를 바탕으로 데이터를 삭제할 수 있습니다.

    dict_['context'] = ' '.join(docs)
    dict_['answers']['answer_start'] = []
    dict_['answers']['text'] = ['']
    dict_['id'] = 'NEG_'+dict_['id']
    return dict_


In [ ]:
m = load_from_disk(path+#데이터셋이름)

In [ ]:
m2 = m.map(full_neg_2_concat)
m2['context'][:5]

['제로-100 프로젝트의 5대 과제는 다음과 같다. # 과제누수 제로를 위해 2012년 정책포털시스템을 구축하고 그 안에 도정과제관리시스템을 구축하여 개인이 추진하고 있는 과제(2014년 말 기준 3,132개)를 등록하여 관리하고 전 직원이 공유한다. #직무수수 제로를 위해 내부 직원들이 사용하는 행정포털시스템에 직무편람시스템을 구축(2012년)하고 조직원들이 업무 추진과정에서 알게된 노하우나 전문지식을 편람으로 등록(2014년 말 기준 3,675건)하여 업무 관련 지식을 체계적으로 축적하고 공유하고 있으며 담당자 변경시 인계인수를 통해 누수 없는 업무를 추진한다. # 회의낭비 제로를 위해 2012년 정책포털에 디지털회의시스템을 구축하고 회의개요, 회의자료, 회의결과 등을 등록·공유하여 하고 있으며 불필요한 회의 축소 등 생산적인 회의문화를 조성한다. # 민원누수 제로를 위해 민원처리과정을 공개하고 2017년 민원통합관리시스템을 구축하여 민원정보를 집적하고 통합관리 및 운영한다. # 충청남도는 국민주권주의를 실현하기 위해서는 정부가 보유한 정보의 공개가 먼저 이루어져야 함을 인식하고 결재문서, 재정정보 등 다양한 정보를 100% 공개하기 위해 노력한다. 2017년 기준 충청남도의 결재문서 원문정보 공개율을 95.5% 로 전 행정기관(공공기관 제외) 중 1위이다. 또한 2013년 전국 최초로 재정정보공개시스템을 구축하고 세입·세출내역 등 살림살이 정보를 내부 재정관리시스템인 e호조와 연계하여 공개하기 시작했고 , 이는 국가재정법 및 지방재정법 개정으로 이어져 전국 행정기관으로 확산되었다. 충청남도는 이에 그치지 않고 2016년 말 도 소속 20개 공공기관이 공동으로 활용하는 공공기관 업무지원시스템을 구축하고 2017년부터 공공기관까지 문서 및 재정정보공개를 시작하였다. 이 외에도, 중장기계획, 통계, 주요 지표의 체계적 관리, 자료취합시스템, 통화이력 관리시스템, 협업포인트 제도, 비영리법인 관리시스템, 재정정보공개시스템 고도화 등 지속적으로 제로-100 프로

In [ ]:
m2.save_to_disk(path+#데이터셋이름)

## 오답 문서 concat 원본 문서-질문 데이터 제작

리트리버가 가져온 후보 문서들을 concat 합니다.

단, answers span이 적절하지 않은 문서에 존재하더라도 그 문서를 concat합니다.

문서 길이는 2000자 까지만 받아 넣습니다.

---

학습 코드에서 데이터 셋을 모델의 입력 차원 만큼으로 잘라서 넣기 때문에, 

실제 학습에서는 위의 방법과 큰 차이는 없습니다.

In [ ]:
def neg_2_concat(dict_): 
    query = {
        'query':{
            'bool':{
                'must':[
                        {'match':{'text':dict_['question']}}
                ],
                'should':[
                        {'match':{'text':' '.join([i[0] for i in ner(dict_['question']) if i[1] != 'O'])}}
                ]
            }
        }
    }
    es_docs = es.search(index='document_a',body=query,size=3)['hits']['hits']
    
    docs = []
    for doc in es_docs:
        d = doc['_source']['text_origin']
        if dict_['context'] == d:
            continue
        docs.append(d[:2000])
        if len(docs) == 2:
            break
    
    dict_['context'] = dict_['context'] + ' ' + ' '.join(docs)
    return dict_


In [ ]:
tdk12_tag_dataset = load_from_disk(path+#데이터 셋 이름)

In [ ]:
tdk12_tag_dataset2 = tdk12_tag_dataset.map(neg_2_concat)
tdk12_tag_dataset2['context'][:5]

['미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05 미국 상의원 또는 미국 상원(united states senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며,

In [ ]:
tdk12_tag_dataset2.save_to_disk(path+#데이터 셋 이름)